In [44]:
import pandas as pd


sightings = pd.read_csv('koala-survey-sightings-data.csv', encoding='utf-8', parse_dates=[['Date', 'Time']])

# They used a shorthand for some of the genus
sightings['TreeSpecies'] = sightings['TreeSpecies'].replace({"E.": "Eucalyptus", "L.":"Lophostemon", "C.": "Corymbia", "A.": "Angophora"}, regex=True)
sightings['TreeSpecies'] = sightings['TreeSpecies'].dropna()
sightings.drop_duplicates(subset=['TreeSpecies'], inplace=True)

sightings[['TreeSpecies']].to_csv('koala-sightings-species.csv')
# ^ Upload that file to the ALA and download the results and field guide with a mapping to the correct species.
#sightings_with_species_info = pd.merge(sightings, unique_df, right_on="Scientific Name - original", left_on="TreeSpecies", how="left")

In [45]:
import requests
from urllib.parse import urlsplit
import os
processed_df = pd.read_csv('SEQ_Koala_Survey_Data_2010_-_2015.csv')

for _, row in processed_df.iterrows():
    doc_id = urlsplit(row["guid"]).path.split('/')[-1]
    r = requests.get(row["guid"] + ".json")
    with open(os.path.join('apni', doc_id + '.json'), 'wb') as f:
        f.write( r.content )

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlsplit
import os

processed_df = pd.read_csv('SEQ_Koala_Survey_Data_2010_-_2015.csv')

for _, row in processed_df.iterrows():
    try:
        doc_id = urlsplit(row["guid"]).path.split('/')[-1]
        path = os.path.join('apni', doc_id + '.txt')
        if not os.path.exists(path):
            response = requests.get(
                url="https://en.wikipedia.org/wiki/{0}".format(row["scientificName"].replace(" ", "_")),
            )
            soup = BeautifulSoup(response.content, 'html.parser')
            d = soup.find(id="Description").parent.next_sibling.next_sibling.get_text()
            with open(os.path.join('apni', doc_id + '.txt'), 'w') as f:
                f.write(d)
        else:
            print(path, " already exists")
    except Exception as e:
        print(e, row[0])

apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/2896580.txt  already exists
apni/28965